In [1]:
import numpy as np
#np.set_printoptions(precision=5, linewidth=200, suppress=True)
import psi4
from helper_PFCI import PFHamiltonianGenerator
from helper_cs_cqed_cis import *

In [2]:
#mol_str = """
#O
#H 1 1.1
#H 1 1.1 2 104
#symmetry c1
#"""

mol_str = """
Li
H 1 1.0
symmetry c1
"""
options_dict = {'basis': 'sto-3g',
                  'scf_type': 'pk',
                  'e_convergence': 1e-10,
                  'd_convergence': 1e-10,
                  }

mol = psi4.geometry(mol_str)


psi4.set_options(options_dict)
psi4.core.set_output_file('output.dat', False)

In [3]:
# photon energy
omega_val = 2.0 / psi4.constants.Hartree_energy_in_eV
#omega_val = 0.0

print(omega_val)

# lambda vector
lambda_vector = np.array([.01, .01, .01])
#lambda_vector = np.array([0, 0, 0.0])

# Build PF Hamiltonian matrix 
H2_PF = PFHamiltonianGenerator(1, mol_str, options_dict, lambda_vector,omega_val, ignore_coupling=False, ci_level="cis")

# uncomment to print
#print(H2_PF.H_PF)


0.07349864496170931

Start SCF iterations:

Canonical RHF One-electron energy = -13.3346745238277329
CQED-RHF One-electron energy      = -13.3338879268738726
Nuclear repulsion energy          = 1.5875316320100001
Dipole energy                     = 0.0004020117084399
SCF Iteration   1: Energy = -7.7668598404831979   dE = -7.76686E+00   dRMS = 6.48590E-05
SCF Iteration   2: Energy = -7.7668600544265285   dE = -2.13943E-07   dRMS = 1.13708E-05
SCF Iteration   3: Energy = -7.7668600673978734   dE = -1.29713E-08   dRMS = 4.90503E-06
SCF Iteration   4: Energy = -7.7668600701681338   dE = -2.77026E-09   dRMS = 2.39509E-06
SCF Iteration   5: Energy = -7.7668600708436610   dE = -6.75527E-10   dRMS = 1.19367E-06
SCF Iteration   6: Energy = -7.7668600710151861   dE = -1.71525E-10   dRMS = 6.05528E-07
SCF Iteration   7: Energy = -7.7668600710603979   dE = -4.52118E-11   dRMS = 3.12866E-07
SCF Iteration   8: Energy = -7.7668600710727542   dE = -1.23563E-11   dRMS = 1.64534E-07
SCF Iteration   9: E

TypeError: calc1RDMElement() missing 1 required positional argument: 'ket_idx'

In [ ]:
e_fci, wavefunctions = np.linalg.eigh(H2_PF.H_PF)
print(e_fci[4]-e_fci[0])
print(e_fci[4])


c_4 = wavefunctions[:,4]
c_4s = np.conj(c_4.T)

E_exp = c_4s @ H2_PF.H_PF @ c_4

norm = c_4.T @ c_4
print(norm)
print(E_exp)
print(c_4)

In [ ]:
n_el_dets = len(H2_PF.dets)
n_el_sings = n_el_dets - 1
n_elp_dets = len(c_4)

print(n_el_dets)
print(n_el_sings)
print(n_elp_dets)

In [ ]:
for i in range(n_el_dets):
    print(F" {c_4s[i]:.4e} : {H2_PF.dets[i]}|0>")
    
    
for i in range(n_el_dets):
    print(F" {c_4s[i+n_el_dets]:.4e} : {H2_PF.dets[i]}|1>")



In [ ]:
from helper_PFCI import Determinant

occList = [i for i in range(H2_PF.ndocc)]
print(occList)

det_ref = Determinant(alphaObtList=occList, betaObtList=occList)
print(det_ref)
alphaO, betaO = det_ref.getOrbitalIndexLists()
alphaU, betaU = det_ref.getUnoccupiedOrbitalsInLists(H2_PF.nmo)
print(alphaU)

In [ ]:
# diagonalize the Hamiltonian matrix
e_fci, wavefunctions = np.linalg.eigh(H2_PF.H_PF)

#print the ground-state and first excited state
#print("Psi4 |G> is ",fci_e)
print("|G> is ",e_fci[0])
print("|LP> is ",e_fci[1])
#|G> is  -75.01298018271805
#|LP> is  -75.01298018271805
assert np.isclose(e_fci[0],-74.94207989868084)

In [ ]:
cqed_cis_dict = cs_cqed_cis(lambda_vector, omega_val, mol_str, options_dict)
_c_vecs = cqed_cis_dict["CQED-CIS L VECTORS"]



In [ ]:
c_0 = _c_vecs[:,0]
c_1 = _c_vecs[:,1]
c_2 = _c_vecs[:,2]
c_3 = _c_vecs[:,3]
c_4 = _c_vecs[:,4]

print(c_4)

In [ ]:
_nmo = H2_PF.nmo
_ndocc = H2_PF.ndocc
_nvirt = _nmo - _ndocc

# total number of states in {|R,0>, |R,1>, |S,0>, |S,1>}
_n_ss = len(c_1)
# total number of states in {|R,0>, |S,0>} and in {|R,1>, |S,1>}
_n_s = int(_n_ss / 2)

# right now the CI configurations are odered like the following:
# |R,0>, |R,1>, |S1,0>, |S1,1>, |S2,0>, |S2,1>, ...
# where |S1> denotes the first singly-excited derminant, |S2> the second, etc
# so the configurations with 0 photon occupation have even indices and
# the configurations with 1 photon occupation have odd indices.

_pvac_idx = np.arange(0, _n_ss, 2)  # <== indices for C coeffs for |0> states
_pone_idx = np.arange(1, _n_ss, 2)  # <== indices for C coeffs for |1> states

# CIS coefficients spanning the |R,0> and |S,0> states
c_n0 = c_1[_pvac_idx]
# CIS coefficients spanning the |R,1> and |R,1> states
c_n1 = c_1[_pone_idx]

# now each vector spans the |R> + {|S>} basis for a given photon state.
# for the {|S>}, let's generate a list of the excitations in the order
# of the |S> basis
_pi = np.zeros(_nmo)
_xi = np.zeros(_nmo)


excitations = []
C0_ia = np.zeros((_nmo, _nmo))
C1_ia = np.zeros((_nmo, _nmo))
_idx = 1
for _i in range(_ndocc):
    _xi[_i] = 1
    for _a in range(_nvirt):
        _A = _a + _ndocc
        _pi[_A] = 1
        excitations.append((_i, _A))
        C0_ia[_i, _A] = c_n0[_idx]
        C1_ia[_i, _A] = c_n1[_idx]
        _idx += 1

print(c_n0)
print(c_n1)
print(np.dot(c_n0, c_n0))
print(np.dot(c_n1, c_n1))

print(_pi)

In [ ]:


_D1 = np.zeros((_nmo, _nmo))
for _i in range(_ndocc):
    _D1[_i, _i] = c_n0[0] * c_n0[0] + c_n1[0] * c_n1[0]

for _p in range(_nmo):
    for _q in range(_nmo):
        _D1[_p, _q] += c_n0[0] * C0_ia[_p, _q] * _pi[_p] * _xi[_q]
        _D1[_p, _q] += c_n0[0] * C0_ia[_p, _q] * _xi[_p] * _pi[_q]
        _D1[_p, _q] += c_n1[0] * C1_ia[_p, _q] * _pi[_p] * _xi[_q]
        _D1[_p, _q] += c_n1[0] * C1_ia[_p, _q] * _xi[_p] * _pi[_q]

        for _i in range(_ndocc):
            _D1[_p, _q] += C0_ia[_p, _i] * C0_ia[_i, _q] * _pi[_p] * _pi[_q]
            _D1[_p, _q] += C1_ia[_p, _i] * C1_ia[_i, _q] * _pi[_p] * _pi[_q]

        for _a in range(_nvirt):
            _A = _a + _ndocc
            _D1[_p, _q] -= C0_ia[_a, _q] * C0_ia[_p, _a] * _xi[_p] * _xi[_q]
            _D1[_p, _q] -= C1_ia[_a, _q] * C1_ia[_p, _a] * _xi[_p] * _xi[_q]


ndocc = 5
n_act_el = 4
n_act_orb = 4

n_in_orb=ndocc-n_act_el//2
n_ac_el_half=n_act_el//2
print(n_ac_el_half)
inactive_list=list(x for x in range(n_in_orb))
print(inactive_list)
print('Generating all determinants in active space')
t = time.time()
singlesDets = []
for alpha in combinations(range(n_act_orb), n_ac_el_half):
    alphalist=list(alpha)
    alphalist=[x+n_in_orb for x in alphalist]
    alphalist[0:0]=inactive_list
    alpha=tuple(alphalist)
    #alpha_ex_level = compute_excitation_level(alpha, ndocc)
    for beta in combinations(range(n_act_orb), n_ac_el_half):
        betalist=list(beta)
        betalist=[x+n_in_orb for x in betalist]
        betalist[0:0]=inactive_list
        beta=tuple(betalist)
        #beta_ex_level = compute_excitation_level(beta, ndocc)
        #if alpha_ex_level + beta_ex_level == 1:
        print(F' adding alpha: {alpha} and beta: {beta}\n') 
        singlesDets.append(Determinant(alphaObtList=alpha, betaObtList=beta))